In [51]:
import pandas as pd
import numpy as np
from prophet import Prophet
from data_extraction.dummy_data_extractor import extract_dummy_data

In [53]:
master_table = extract_dummy_data("dummy_data")

c:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\models\Final Models\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-05-09 23:55:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [54]:
master_table

,pointID,unit,dqType,dqStart,dqDuration,pointInterval,his
0,@p:dmc_All:r:2e223a3a-5053d2b0 DMC Building 1 ...,°C,Nulls,2023-05-09 23:55:00+04:00,2 days 00:05:00,0 days 00:05:00,Timestamp val \ 0...


In [55]:
df = master_table.at[0, "his"].iloc[:, :2]

In [56]:
df.set_index(df.columns[0], inplace=True, drop=True)

In [57]:
 df = df[df.index < dqStart]

In [58]:
df.reset_index(inplace=True)

In [30]:
dqStart 

Timestamp('2023-05-09 23:55:00+0400', tz='UTC+04:00')

In [60]:
# Rename columns for convenience
df.columns = ['ds','y']
df

,ds,y
0,2023-04-09 13:20:00+04:00,17.938112
1,2023-04-09 13:25:00+04:00,17.938112
2,2023-04-09 13:30:00+04:00,17.938112
3,2023-04-09 13:35:00+04:00,17.938112
4,2023-04-09 13:40:00+04:00,17.938112
...,...,...
8762,2023-05-09 23:30:00+04:00,16.263992
8763,2023-05-09 23:35:00+04:00,16.114820
8764,2023-05-09 23:40:00+04:00,16.114820
8765,2023-05-09 23:45:00+04:00,16.114820


In [61]:
df['ds'] = df['ds'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [62]:
# Initialize Prophet model with tuned hyperparameters
model_temp = Prophet(seasonality_mode='additive',  # Adjust based on data exploration
                         interval_width=0.95,          # Adjust prediction interval if needed
                         changepoint_prior_scale=0.01) # Tune based on data patterns


In [63]:
 # Fit the model
model_temp.fit(df)

09:35:48 - cmdstanpy - INFO - Chain [1] start processing
09:35:50 - cmdstanpy - INFO - Chain [1] done processing


In [64]:
length_of_missing_data = pd.Timedelta(master_table.at[0, "dqDuration"])
data_logging_interval = pd.Timedelta(master_table.at[0, "pointInterval"])

In [65]:
samples = int(length_of_missing_data / data_logging_interval) + 1
samples

578

In [67]:
# Create future DataFrame
future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

In [69]:
# Predict the future values
forecast_temp = model_temp.predict(future_temp)


In [70]:
forecast_temp

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-04-09 13:20:00,18.379882,16.914839,19.520542,18.379882,18.379882,-0.249702,-0.249702,-0.249702,-0.093936,-0.093936,-0.093936,-0.155767,-0.155767,-0.155767,0.0,0.0,0.0,18.130179
1,2023-04-09 13:25:00,18.377379,16.801051,19.401655,18.377379,18.377379,-0.246315,-0.246315,-0.246315,-0.090790,-0.090790,-0.090790,-0.155525,-0.155525,-0.155525,0.0,0.0,0.0,18.131065
2,2023-04-09 13:30:00,18.374877,16.852163,19.528094,18.374877,18.374877,-0.242482,-0.242482,-0.242482,-0.087211,-0.087211,-0.087211,-0.155271,-0.155271,-0.155271,0.0,0.0,0.0,18.132395
3,2023-04-09 13:35:00,18.372375,16.864903,19.443899,18.372375,18.372375,-0.238246,-0.238246,-0.238246,-0.083240,-0.083240,-0.083240,-0.155006,-0.155006,-0.155006,0.0,0.0,0.0,18.134129
4,2023-04-09 13:40:00,18.369873,16.801979,19.435537,18.369873,18.369873,-0.233651,-0.233651,-0.233651,-0.078923,-0.078923,-0.078923,-0.154728,-0.154728,-0.154728,0.0,0.0,0.0,18.136223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9340,2023-05-11 23:40:00,16.101095,15.135820,17.728063,15.955699,16.241006,0.266499,0.266499,0.266499,0.202887,0.202887,0.202887,0.063612,0.063612,0.063612,0.0,0.0,0.0,16.367594
9341,2023-05-11 23:45:00,16.101055,15.028711,17.668885,15.955334,16.241382,0.275765,0.275765,0.275765,0.213674,0.213674,0.213674,0.062091,0.062091,0.062091,0.0,0.0,0.0,16.376820
9342,2023-05-11 23:50:00,16.101016,14.951287,17.827400,15.954844,16.241758,0.284230,0.284230,0.284230,0.223659,0.223659,0.223659,0.060570,0.060570,0.060570,0.0,0.0,0.0,16.385245
9343,2023-05-11 23:55:00,16.100976,15.047622,17.684849,15.954353,16.242134,0.291868,0.291868,0.291868,0.232817,0.232817,0.232817,0.059051,0.059051,0.059051,0.0,0.0,0.0,16.392844


In [73]:
dq_start = dqStart.strftime('%Y-%m-%d %H:%M:%S')

In [75]:
# Filter predictions to start from dq_start
predictions = forecast_temp[forecast_temp['ds'] >= dq_start][['ds', 'yhat']]

In [76]:
predictions

,ds,yhat
8767,2023-05-09 23:55:00,16.115048
8768,2023-05-10 00:00:00,16.122850
8769,2023-05-10 00:05:00,16.129800
8770,2023-05-10 00:10:00,16.135881
8771,2023-05-10 00:15:00,16.141079
...,...,...
9340,2023-05-11 23:40:00,16.367594
9341,2023-05-11 23:45:00,16.376820
9342,2023-05-11 23:50:00,16.385245
9343,2023-05-11 23:55:00,16.392844


In [78]:
# Set 'ds' as the index
predictions.set_index('ds', inplace=True)

In [ ]:
import pandas as pd
from prophet import Prophet
from data_extraction.dummy_data_extractor import extract_dummy_data

def facebook_pred(df, length_of_missing_data, data_logging_interval, dqStart):
    df.reset_index(inplace=True)
    df = df.dropna()

    # Keep only the first two columns
    df = df.iloc[:, :2]

    # Rename columns
    df.columns = ['ds', 'temp']

    # Remove ' Dubai' from the datetime strings
    df['ds'] = df['ds'].str.replace(' Dubai', '', regex=False)

    # Convert the 'ds' column to datetime format
    df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")

    # Drop rows where datetime parsing failed
    df = df.dropna(subset=['ds'])

    # Clean temperature column and convert to numeric
    df['temp'] = df['temp'].str.replace('°C', '').astype(float)

    # Rename columns for convenience
    df.columns = ['ds', 'y']

    # Ensure 'ds' column is timezone-naive
    df['ds'] = df['ds'].dt.tz_localize(None)

    # Initialize Prophet model with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive',  # Adjust based on data exploration
                         interval_width=0.95,          # Adjust prediction interval if needed
                         changepoint_prior_scale=0.01) # Tune based on data patterns

    # Fit the model
    model_temp.fit(df)

    # Number of predictions
    samples = int(length_of_missing_data / data_logging_interval) + 1

    # Create future DataFrame
    future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)

    # Convert dq_start to timezone-naive
    dq_start = pd.Timestamp(dqStart).tz_convert('Asia/Dubai').tz_localize(None)

    # Ensure 'ds' column in forecast_temp is timezone-naive
    forecast_temp['ds'] = forecast_temp['ds'].dt.tz_localize(None)

    # Filter predictions to start from dq_start
    predictions = forecast_temp[forecast_temp['ds'] >= dq_start][['ds', 'yhat']]

    # Set 'ds' as the index
    predictions.set_index('ds', inplace=True)

    return predictions

In [ ]:
# Example usage:
master_table = extract_dummy_data("dummy_data")
df = master_table.at[1, "his"].iloc[:, :2].copy()

# Extract values from the second row of master_table
length_of_missing_data = pd.Timedelta(master_table.at[1, "dqDuration"])
data_logging_interval = pd.Timedelta(master_table.at[1, "pointInterval"])
dqStart = master_table.at[1, "dqStart"]

# Call the function
predictions = xgboost_1(df, length_of_missing_data, data_logging_interval, dqStart)

# Display the resulting dataframe
print(predictions.head())